In [1]:
import numpy as np
import ipywidgets as iw
from brepmatching.data import BRepMatchingDataset, load_data
import torch
import meshplot
from IPython.display import clear_output, display, HTML

In [2]:
data_path = '/fast/jamesn8/brepmatching/TopoV2FullRun.zip'
data_cache = '/fast/jamesn8/brepmatching/TopoV2FullRun.pt'

cache_data = load_data(data_path, data_cache)
data = BRepMatchingDataset(cache_data, mode='test') # TODO: make sure it's using the correct set/groups

In [13]:
out = iw.Output(layout={'border': '1px solid black'})

@out.capture(clear_output=True, wait=True)
def process_sample(data_ind):
    brep = data[data_ind]
    indices = {}
    for topo_type in ['faces', 'edges', 'vertices']:
        num_right_topos = getattr(brep, 'right_' + topo_type).shape[0]
        exact_matches = getattr(brep, 'bl_exact_' + topo_type + '_matches')
        right_matchless_mask = torch.ones(num_right_topos, dtype=torch.int64)
        right_matchless_mask[exact_matches[1]] = 0
        #TODO: Also mask out the ones for which predictions agree (once you have those)
        #TODO: Baseline heuristic (how to obtain this?)
        indices[topo_type] = right_matchless_mask.nonzero()

    plot_left = meshplot.plot(brep.left_V.numpy(), brep.left_F.T.numpy())
    plot_right = meshplot.plot(brep.right_V.numpy(), brep.right_F.T.numpy())
    display(plot_left)
    display(plot_right)
    #html_left = plot_left.to_html(imports=True, html_frame=False)
    #html_right = plot_right.to_html(imports=True, html_frame=False)
    #display(HTML(html_left))
    #display(HTML(html_right))



In [14]:
curr_ind = 0

button = iw.Button(description='Next')
def on_button_clicked(b):
    global curr_ind
    #global button

    #out.clear_output()
    #display(button)

    curr_ind += 1 #TODO: Start correctly
    process_sample(curr_ind)

button.on_click(on_button_clicked)
display(button)
display(out)

Button(description='Next', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…